In [5]:
import os
import json
import pandas as pd
import traceback


ModuleNotFoundError: No module named 'pandas'

In [6]:
from langchain_openai import ChatOpenAI


In [7]:
import os
key=os.getenv('OPEN_API_KEY')

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
llm = ChatOpenAI(
    openai_api_key=key,  # ✅ correct argument name
    model="gpt-4o-mini",
    temperature=0.5
)